In [2]:
import pyvista as pv
import numpy as np

In [6]:
! zenodo_get 15588103 -g "sUbend.vtu"

Title: Motion in KomaMRI MRM Phantoms
Keywords: 
Publication date: 2025-06-04
DOI: 10.5281/zenodo.15588103
Total size: 114.0 MB

File: sUbend.vtu (114.0 MB)
Link: https://zenodo.org/api/records/15588103/files/sUbend.vtu/content
100% [..................................................] 113976401 / 113976401
Checksum is correct for sUbend.vtu. (a36182d6cf0434fe14279bfef6bd466d)

All specified files have been processed.


In [2]:
def sample_pixel(mesh, x, y, z, pixel_size=1e-3):
    """
    Sample velocity data from a given mesh at a specified pixel location.

    Parameters:
    mesh (pyvista.core.pointset.UnstructuredGrid): The mesh to sample from.
    x (float): The x-coordinate of the pixel's location.
    y (float): The y-coordinate of the pixel's location.
    z (float): The z-coordinate of the pixel's location.
    pixel_size (float, optional): The size of the pixel. Default is 1e-3.

    Returns:
    tuple: Three arrays containing the sampled velocity components (vx, vy, vz).
    """
    e_x = e_y = e_z = pixel_size
    dx , dy, dz = x, y, z
    pixel = pv.Box(bounds = (0, e_x, 0, e_y, 0, e_z), level=0, quads=True).translate([dx-pixel_size/2, dy-pixel_size/2, dz-pixel_size/2])

    # Sample velocity data
    sampled_points = pixel.sample(mesh)
    return sampled_points['U'][:, 0], sampled_points['U'][:, 1], sampled_points['U'][:, 2], pixel

In [3]:
mesh = pv.read('sUbend.vtu')

In [4]:
vx, vy, vz, pixel_mesh = sample_pixel(mesh, 0.1, 0, 0, pixel_size=1e-3)
print(vx.mean(), vy.mean(), vz.mean())

3.2093268104363233 -0.0859851012355648 0.20601526383597957


In [15]:
# pixel_mesh.save('pixel.vtk')

In [5]:
data = np.load("pixel_positions_axial_mask_40x80.npz")
vx_arr = []
vy_arr = []
vz_arr = []

In [6]:
for i in range(len(data['x'])):
    vx, vy, vz, pixel_mesh = sample_pixel(mesh, data['x'][i], data['y'][i], data['z'][i], pixel_size=1e-3)
    vx_arr.append(vx.mean())
    vy_arr.append(vy.mean())
    vz_arr.append(vz.mean())

In [11]:
print("mean: ", np.mean(vx_arr))
print("median: ", np.median(vx_arr))
print("std: ", np.std(vx_arr))
print("min: ", np.min(vx_arr))
print("max: ", np.max(vx_arr))

mean:  0.027650544998626646
median:  -0.48234846201557957
std:  1.094472268638378
min:  -1.070285011199303
max:  4.52255232911557


In [12]:
np.savez("velocity_data_axial_40x80.npz", vx=vx_arr, vy=vy_arr, vz=vz_arr)